In [3]:
import numpy as np
import heapq
import math
import random


def compute_entropy(data, is_probabilities=False):
    """
    Computes the entropy of a dataset, either from a list of realizations or directly from a list of probabilities.

    Parameters:
        data (list):
            - If `is_probabilities` is False: List of realizations of a random variable.
            - If `is_probabilities` is True: List of probabilities associated with unique values.
        is_probabilities (bool): Flag to indicate whether `data` is a list of probabilities (True) or realizations (False).

    Returns:
        entropy (float): The entropy of the dataset.
    """
    if is_probabilities:
        probabilities = np.array(data)
    else:
        # If data is a list of realizations, compute probabilities from frequencies
        unique_values, counts = np.unique(data, return_counts=True)
        probabilities = counts / len(data)

    # Print unique values and probabilities for debugging purposes
    # print("Probabilities:", probabilities)

    # Compute entropy using the formula H(X) = -Σ p(x) log2(p(x))
    entropy = -np.sum(probabilities * np.log2(probabilities + np.finfo(float).eps))
    return entropy
    

In [4]:
# Clase Nodo para representar cada símbolo en el árbol de Huffman
class Nodo:
    def __init__(self, simbolo=None, frecuencia=None):
        self.simbolo = simbolo
        self.frecuencia = frecuencia
        self.izquierda = None
        self.derecha = None

    # Método para comparar nodos según su frecuencia (necesario para la cola de prioridad)
    def __lt__(self, otro):
        return self.frecuencia < otro.frecuencia

In [6]:
def build_huffman_tree(simbolos, valores, is_probabilities=False):
    """
    Build a Huffman tree from symbols and frequencies (or probabilities)

    Parámetros:
        simbolos (list): Lista de símbolos.
        valores (list): Lista de frecuencias o probabilidades asociadas a los símbolos.
        is_probabilities (bool): Indica si los valores son probabilidades (True) o frecuencias (False).

    Retornos:
        Nodo: Raíz del árbol de Huffman.
    """
    if is_probabilities:
        # Validar que las probabilidades sumen aproximadamente 1
        if not (0.99 <= sum(valores) <= 1.01):
            raise ValueError("Las probabilidades deben sumar aproximadamente 1.")

    # Crear una cola de prioridad inicial con nodos para cada símbolo
    cola_prioridad = [Nodo(simbolo, valor) for simbolo, valor in zip(simbolos, valores)]
    heapq.heapify(cola_prioridad)  # Convertir la lista en una cola de prioridad

    # Combinar nodos hasta que quede un único árbol
    while len(cola_prioridad) > 1:
        # Extraer los dos nodos con menor frecuencia
        hijo_izquierdo = heapq.heappop(cola_prioridad)
        hijo_derecho = heapq.heappop(cola_prioridad)
        # Crear un nuevo nodo combinando las frecuencias
        nodo_combinado = Nodo(frecuencia=hijo_izquierdo.frecuencia + hijo_derecho.frecuencia)
        nodo_combinado.izquierda = hijo_izquierdo
        nodo_combinado.derecha = hijo_derecho
        heapq.heappush(cola_prioridad, nodo_combinado)  # Añadir el nuevo nodo a la cola

    # Retornar la raíz del árbol
    return cola_prioridad[0]


In [7]:
def generate_huffman_codes(nodo, codigo="", codigos_huffman={}):
    if nodo is not None:
        if nodo.simbolo is not None:
            codigos_huffman[nodo.simbolo] = codigo
        # Recursivamente generar códigos para los hijos izquierdo y derecho
        generate_huffman_codes(nodo.izquierda, codigo + "0", codigos_huffman)
        generate_huffman_codes(nodo.derecha, codigo + "1", codigos_huffman)

    return codigos_huffman

In [8]:
def compute_average_code_length(probabilities, codes):
    """
    Calculate the average code length of a given set of probabilities and codes

    Parameters:
        probabilities (list): List of probabilities for each symbol.
        codes (list): List of codewords (strings) for each symbol.

    Returns:
        float: The average code length.
    """
    if len(probabilities) != len(codes):
        raise ValueError("The length of probabilities and codes must be the same.")

    # Compute the average code length
    avg_length = sum(p * len(c) for p, c in zip(probabilities, codes))
    return avg_length

# Función para decodificar un mensaje codificado con Huffman usando el árbol de Huffman
def decodificar_huffman(mensaje_codificado, raiz):
    mensaje_decodificado = ""
    nodo_actual = raiz

    for bit in mensaje_codificado:
        # Recorrer el árbol según el bit actual
        if bit == '0':
            nodo_actual = nodo_actual.izquierda
        else:
            nodo_actual = nodo_actual.derecha

        # Si llegamos a un nodo hoja, agregar el símbolo al mensaje decodificado
        if nodo_actual.izquierda is None and nodo_actual.derecha is None:
            mensaje_decodificado += nodo_actual.simbolo
            nodo_actual = raiz  # Reiniciar para el siguiente símbolo

    return mensaje_decodificado

def show_huffman_codes(huffman_codes):
  print("Huffman codes generated")
  for symbol, h_code in huffman_codes.items():
    print(f"Symbol: {symbol}, Code: {h_code}")

def order_dict_alphabetically(dict_codes):
  hc = dict(sorted(dict_codes.items())).copy()
  return hc

def get_list_symbols_codes(dict_codes):
  return list(dict_codes.keys()), list(dict_codes.values())

def generate_equal_probabilities(size):
    """
    Generate a list of equal probabilities for a given size.

    Parameters:
        size (int): The size of the list for which probabilities are generated.

    Returns:
        list: A list of probabilities summing to 1, with each element equal to 1/size.
    """
    if size <= 0:
        raise ValueError("The size of the list must be greater than 0.")

    probability = 1 / size
    return [probability] * size

In [10]:
# Función para agregar errores aleatorios al mensaje codificado
def agregar_errores_aleatorios(mensaje_codificado, tasa_error=0.1):
    vector_error = ''.join('1' if random.random() < tasa_error else '0' for _ in mensaje_codificado)
    # Realizar XOR entre el mensaje codificado y el vector de error para alterar bits
    mensaje_alterado = ''.join('1' if mensaje_codificado[i] != vector_error[i] else '0' for i in range(len(mensaje_codificado)))
    return mensaje_alterado, vector_error

# Códigos Huffman

In [11]:
# --- PROGRAMA PRINCIPAL ---

frase = "abcdfeacbdefabcdefadcbefcbbdaef"
simbolos = ['a', 'b', 'c', 'd', 'e', 'f']
frecuencias = [0] * len(simbolos)

for c in frase:
    for j, val in enumerate(simbolos):
        if (c == val):
            frecuencias[j] += 1

# 1. Construir el árbol y generar códigos
raiz = build_huffman_tree(simbolos, frecuencias)
codigos_huffman = generate_huffman_codes(raiz)

print("Códigos Huffman:")
for simbolo, codigo in codigos_huffman.items():
    print(f"Símbolo: {simbolo}, Código: {codigo}")

# 2. Cálculos Estadísticos
probabilidades = np.array(frecuencias) / len(frase)

# Longitud promedio Huffman (sum p_i * L_i)
longitud_huffman = sum(probabilidades[i] * len(codigos_huffman[simbolos[i]]) for i in range(len(simbolos)))

# Codificación de longitud fija (L = ceil(log2(N)))
longitud_fija = np.ceil(np.log2(len(simbolos)))

# Calcular la entropía de la fuente (H = -sum p*log2(p))
eps = 1e-12
entropia = -np.sum(probabilidades * np.log2(probabilidades + eps))

# 3. Comparación y conclusión
print("\nComparación:")
print(f"Longitud promedio Huffman: {longitud_huffman:.4f}")
print(f"Longitud fija: {longitud_fija:.4f}")
print(f"Entropía: {entropia:.4f}")
print(f"La longitud promedio de Huffman ({longitud_huffman:.2f}) está más cerca de la entropía ({entropia:.2f}).")

# 4. Codificar, agregar errores y decodificar
mensaje_codificado = ''.join([codigos_huffman[simbolo] for simbolo in frase])
print("\nMensaje codificado (Huffman):", mensaje_codificado)

tasa_error = 0.1  
mensaje_alterado, vector_error = agregar_errores_aleatorios(mensaje_codificado, tasa_error)
print("\nMensaje alterado (Huffman):  ", mensaje_alterado)

Códigos Huffman:
Símbolo: d, Código: 00
Símbolo: b, Código: 01
Símbolo: f, Código: 100
Símbolo: c, Código: 101
Símbolo: a, Código: 110
Símbolo: e, Código: 111

Comparación:
Longitud promedio Huffman: 2.6452
Longitud fija: 3.0000
Entropía: 2.5814
La longitud promedio de Huffman (2.65) está más cerca de la entropía (2.58).

Mensaje codificado (Huffman): 1100110100100111110101010011110011001101001111001100010101111100101010100110111100

Mensaje alterado (Huffman):   1100110100000111110101010011110111001101001111001110010110111101101010100110011100


In [10]:
#Repetir el apartado anterior considerando el siguiente mensaje "Que es eso eso es queso"

In [13]:
import numpy as np

# 1. Configuración de la nueva frase
frase = "Que es eso eso es queso"
# Creamos el alfabeto dinámicamente para incluir espacios y mayúsculas
simbolos = sorted(list(set(frase))) 
frecuencias = [0] * len(simbolos)

# 2. Conteo de frecuencias [cite: 5, 15]
for c in frase:
    for j, val in enumerate(simbolos):
        if (c == val):
            frecuencias[j] += 1

# 3. Construcción del árbol y generación de códigos 
raiz = build_huffman_tree(simbolos, frecuencias)
codigos_huffman = generate_huffman_codes(raiz, codigo="", codigos_huffman={})

# 4. Cálculos Estadísticos 
probabilidades = np.array(frecuencias) / len(frase) 

# Longitud promedio Huffman (L = sum p_i * n_i) 
longitud_huffman = sum(probabilidades[i] * len(codigos_huffman[simbolos[i]]) for i in range(len(simbolos)))

# Codificación de longitud fija 
longitud_fija = np.ceil(np.log2(len(simbolos)))

# Entropía de la fuente [cite: 2, 10, 15]
eps = 1e-12
entropia = -np.sum(probabilidades * np.log2(probabilidades + eps)) 

print("Códigos Huffman:")
for simbolo, codigo in codigos_huffman.items():
    print(f"Símbolo: {simbolo}, Código: {codigo}")
print("")

# --- RESULTADOS ---
print(f"Frase: '{frase}'")
print("-" * 30)
print(f"Longitud Fija: {longitud_fija} bits")
print(f"Longitud Huffman: {longitud_huffman:.4f} bits")
print(f"Entropía (H): {entropia:.4f} bits")
print("-" * 30)
print(f"Eficiencia: {(entropia/longitud_huffman)*100:.2f}%")

Códigos Huffman:
Símbolo: s, Código: 00
Símbolo:  , Código: 01
Símbolo: e, Código: 10
Símbolo: o, Código: 110
Símbolo: u, Código: 1110
Símbolo: q, Código: 11110
Símbolo: Q, Código: 11111

Frase: 'Que es eso eso es queso'
------------------------------
Longitud Fija: 3.0 bits
Longitud Huffman: 2.5652 bits
Entropía (H): 2.5460 bits
------------------------------
Eficiencia: 99.25%


In [14]:
alfabeto_x1 = [-7, -5, -3, -1, 0, 1, 3, 5, 7]
alfabeto_x2 = [-7, -5, -3, -1, 0, 1, 3, 5, 7]

probabilidades_x1 = [0.02, 0.05, 0.08, 0.15, 0.25, 0.15, 0.10, 0.12, 0.08]
probabilidades_x2 = [0.04, 0.06, 0.10, 0.14, 0.22, 0.18, 0.10, 0.10, 0.06]

huffman_tree_x1 = build_huffman_tree(alfabeto_x1, probabilidades_x1, is_probabilities=True)
huffman_codes_x1 = generate_huffman_codes(huffman_tree_x1)

huffman_tree_x2 = build_huffman_tree(alfabeto_x2, probabilidades_x2, is_probabilities=True)
huffman_codes_x2 = generate_huffman_codes(huffman_tree_x2)

print("Huffman codes for x1:")
show_huffman_codes(huffman_codes_x1)

print("Huffman codes for x2:")
show_huffman_codes(huffman_codes_x2)

longitud_media_x1 = compute_average_code_length(probabilidades_x1, list(huffman_codes_x1.values()))
longitud_media_x2 = compute_average_code_length(probabilidades_x2, list(huffman_codes_x2.values()))

print(longitud_media_x1)
print(longitud_media_x2)

entropia_x1 = compute_entropy(probabilidades_x1, is_probabilities=True)
entropia_x2 = compute_entropy(probabilidades_x2, is_probabilities=True)

print(entropia_x1)
print(entropia_x2)

eficiencia_x1 = entropia_x1 / longitud_media_x1
eficiencia_x2 = entropia_x2 / longitud_media_x2

print(eficiencia_x1)
print(eficiencia_x2)

alfabeto_cuantizado_x1 = [-6, -2, 2, 6, 0]
alfabeto_cuantizado_x2 = [-6, -2, 2, 6, 0]

probabilidades_cuantizado_x1 = [0.07, 0.23, 0.4, 0.22, 0.08]
probabilidades_cuantizado_x2 = [0.1, 0.24, 0.4, 0.2, 0.06]

alfabeto_x1 = alfabeto_cuantizado_x1
alfabeto_x2 = alfabeto_cuantizado_x2

probabilidades_x1 = probabilidades_cuantizado_x1
probabilidades_x2 = probabilidades_cuantizado_x2


huffman_tree_x1 = build_huffman_tree(alfabeto_x1, probabilidades_x1, is_probabilities=True)
huffman_codes_x1 = generate_huffman_codes(huffman_tree_x1)

huffman_tree_x2 = build_huffman_tree(alfabeto_x2, probabilidades_x2, is_probabilities=True)
huffman_codes_x2 = generate_huffman_codes(huffman_tree_x2)

print("Huffman codes for x1:")
show_huffman_codes(huffman_codes_x1)

print("Huffman codes for x2:")
show_huffman_codes(huffman_codes_x2)

longitud_media_x1 = compute_average_code_length(probabilidades_x1, list(huffman_codes_x1.values()))
longitud_media_x2 = compute_average_code_length(probabilidades_x2, list(huffman_codes_x2.values()))

print(longitud_media_x1)
print(longitud_media_x2)

entropia_x1 = compute_entropy(probabilidades_x1, is_probabilities=True)
entropia_x2 = compute_entropy(probabilidades_x2, is_probabilities=True)

print(entropia_x1)
print(entropia_x2)

eficiencia_x1 = entropia_x1 / longitud_media_x1
eficiencia_x2 = entropia_x2 / longitud_media_x2

print(eficiencia_x1)
print(eficiencia_x2)

entropia_conjunta = 0.4 * entropia_x1 + 0.6 * entropia_x2

Huffman codes for x1:
Huffman codes generated
Symbol: d, Code: 00
Symbol: b, Code: 01
Symbol: f, Code: 100
Symbol: c, Code: 101
Symbol: a, Code: 110
Symbol: e, Code: 111
Symbol: 7, Code: 1001
Symbol: 3, Code: 1101
Symbol: 0, Code: 01
Symbol: 5, Code: 001
Symbol: 1, Code: 111
Symbol: -1, Code: 101
Symbol: -7, Code: 1000
Symbol: -5, Code: 1100
Symbol: -3, Code: 000
Huffman codes for x2:
Huffman codes generated
Symbol: d, Code: 00
Symbol: b, Code: 01
Symbol: f, Code: 100
Symbol: c, Code: 101
Symbol: a, Code: 110
Symbol: e, Code: 111
Symbol: 7, Code: 1001
Symbol: 3, Code: 1101
Symbol: 0, Code: 01
Symbol: 5, Code: 001
Symbol: 1, Code: 111
Symbol: -1, Code: 101
Symbol: -7, Code: 1000
Symbol: -5, Code: 1100
Symbol: -3, Code: 000


ValueError: The length of probabilities and codes must be the same.